In [1]:
!pip install konlpy

     ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/19.4 MB ? eta -:--:--
     --------------------------------------- 0.0/19.4 MB 435.7 kB/s eta 0:00:45
     --------------------------------------- 0.1/19.4 MB 751.6 kB/s eta 0:00:26
     ----- ---------------------------------- 2.5/19.4 MB 15.8 MB/s eta 0:00:02
     ------------- -------------------------- 6.7/19.4 MB 30.8 MB/s eta 0:00:01
     -------------------- ------------------ 10.1/19.4 MB 38.0 MB/s eta 0:00:01
     --------------------------- ----------- 13.6/19.4 MB 73.1 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 MB 72.6 MB/s eta 0:00:01
     ----------------------------- --------- 14.7/19.4 M

In [2]:
!pip install Twitter

  Using cached twitter-1.19.6-py2.py3-none-any.whl (50 kB)


In [3]:
!pip install tqdm

  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


In [4]:
!pip install gensim

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/ab/b0/d58dc405fd60ab546ca714321235dc2d455b2dc06bfb4fc1092940c749fc/gensim-4.3.2-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.7.0 from https://files.pythonhosted.org/packages/70/03/485f73046134400ea25d3cb178c5e6728f9b165f79d09638ecb44ee0e9b1/scipy-1.11.2-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ------------------- ------------------ 30.7/59.1 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 59.1/59.1 kB 775.6 kB/s eta 0:00:00
  Obtaining dependency information for smart-open>=1.8.1 from https://files.pythonhosted.org/packages/fc/d9/d97f1db64b09278aba64e8c81b5d322d436132df5741c518f3823824fae0/smart_open-6.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/24.0 MB ? eta -:--

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
theano-pymc 1.1.2 requires filelock, which is not installed.
arviz 0.12.1 requires matplotlib>=3.0, which is not installed.
arviz 0.12.1 requires xarray>=0.16.1, which is not installed.
xarray-einstats 0.5.1 requires xarray>=2022.09.0, which is not installed.


In [5]:
import os
import pymysql
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from gensim.models.word2vec import Word2Vec

In [6]:
# 본인 컴퓨터의 자바 설치 경로 설정
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"

In [7]:
os.environ["JAVA_HOME"]

'C:\\Program Files\\Java\\jdk-11'

In [8]:
from konlpy.tag import Twitter

In [9]:
# 한국어 문자열을 단어로 나누고 품사를 리턴하는 Twitter() 객체 생성
twitter = Twitter()

c:\Users\tjk\AppData\Local\anaconda3\envs\movie_env01\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [10]:
twitter.pos("안녕 하십니까 좋은 아침입니다. 반가워")

[('안녕', 'Noun'),
 ('하십니까', 'Verb'),
 ('좋은', 'Adjective'),
 ('아침', 'Noun'),
 ('입니다', 'Adjective'),
 ('.', 'Punctuation'),
 ('반가워', 'Adjective')]

In [11]:
# stem=True: 동사인 하십니까 반가워는 기본 동사형 '안녕하다'와 '반갑다'로 변환
twitter.pos("안녕 하십니까 좋은 아침입니다. 반가워 집으로 뛰어 갔다 정말 참으로 좋다", stem=True)

[('안녕', 'Noun'),
 ('하다', 'Verb'),
 ('좋다', 'Adjective'),
 ('아침', 'Noun'),
 ('이다', 'Adjective'),
 ('.', 'Punctuation'),
 ('반갑다', 'Adjective'),
 ('집', 'Noun'),
 ('으로', 'Josa'),
 ('뛰다', 'Verb'),
 ('가다', 'Verb'),
 ('정말', 'Noun'),
 ('참으로', 'Noun'),
 ('좋다', 'Adjective')]

In [13]:
# 텍스트 전처리 함수
# text는 전처리할 영화 줄거리(synopsys)입니다.
def preprocessingText(text):
    # 전처리 결과를 저장할 리스트
    stems = []
    
    # text를 단어로 나누고 품사를 붙여서 리턴
    tagged_review = twitter.pos(text, stem=True)
    
    # tagged_review에서 하나를 꺼내서 word, pos에 대입
    # 단어는 word에, 품사는 pos에 대입
    for word, pos in tagged_review:
        if pos == "Noun" or pos == "Adjective":
            stems.append(word)
            
    return " ".join(stems)

In [14]:
# 함수 테스트
preprocessingText("아버지가 방에 들어가신다 그래서 슬펐다 배가 아파요")

'아버지 방 슬프다 배 아프다'

In [15]:
# DB 연결
db = pymysql.connect(
    host="database-1.crcwt5epkeh5.ap-northeast-2.rds.amazonaws.com",
    port=3306,
    user='admin',
    passwd='password',
    db='movie_db',
    charset='utf8'
)